Clarifications and questions for hw3:
use any of the datasets can also use the semeval copied into the files section
part1:
identify slang words and print in CSV, do not need to do replacements in text, oneline for each tweet, original tweet, slang word->replacement, slang_word->replacement

part2: replace one at a time with a synonym, empty cell if less than 5 augments fill these with "None" each entry in the CSV 

part3: dont worry about output of part 2 in part 2 have 10 sentences, 30 variations generated, augmented set has 40 sentences, find how many positive how many negative for ratio
part 4 augmented data needed here
part 5 no augmented data write in a text file the types of differences observed
part6: build a word vector build sentence vectors just do whatever demonstrate effective usage of vectorizer, just work on the original data set

In [205]:
import nltk
import pandas as pd
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from nltk.tokenize import TweetTokenizer
import re
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import copy
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Import Semeval dataset (https://www.kaggle.com/azzouza2018/semevaldatadets) using Python.

Do the following for the Semeval dataset mentioned above using python:

- (1) Remove all URLs from the sentences

- (1) Remove sentences which have less than 4 tokens

- (1) Remove the tokens which starts with @

- (1) Remove all hashtags

- (2) Remove occurrences of characters that appear more than 3 times consecutively, keep max occurrence as 3. For example: Heeeeelooooo => Heeelooo, gooooo => gooo 

- (3) Identify the slang words using a slang dictionary (e.g.: https://github.com/rishabhverma17/sms_slang_translator/blob/master/slang.txt) and output as follows in a CSV file:

      original tweet, slang word -> replacement word, .....

In [206]:
#part1
#note am using semeval-2014-test.csv 
data = pd.read_csv('semeval-2014-test.csv', delimiter='\t')
slang = pd.read_csv('slang.csv', delimiter='=')
slang_dict = {}
with open('slang.csv', newline = '') as csvfile:
    reader = csv.reader(csvfile, delimiter = '=')
    for row in reader:
        #print (row)
        slang_dict[row[0]] = row[1]
#slang_dict["You"] = "testing"
#print (slang_dict)
data.describe()
tkn = TweetTokenizer()
# fix length
text = [tkn.tokenize(tweet) for tweet in data['text']]
labels =  data['label']
min_len = []
for i in range (0, len(text)):
  if len(text[i]) > 3:
    min_len.append(text[i])
text = min_len


#filtering with regexes
filtered = []
with open('slang_words.csv', 'w', newline = '') as csvfile:
  writer = csv.writer(csvfile, delimiter = ',', lineterminator='\n')
  writer.writerow(['Original Tweet', 'Slang Words'])
  for tweet in text:
    temp = copy.deepcopy(tweet)
    slang_out = []
    for token in tweet:
      if re.match("https?\://", token):
        temp.remove(token)
      if re.match("#([a-zA-Z0-9_])", token):
        temp.remove(token)
      if re.match("@", token):
        temp.remove(token)
        #print (token)
      #handle slang

      # handles slang
      for key, val in slang_dict.items():
        if key == token:
          print (key)
          #export to csv
          slang_out.append(key + "->" + val)
          #temp[temp.index(token)] = val
    for token in temp:
     # temp1 = re.compile(r'([a-zA-Z])\1\1+')
      #if temp1.findall(token) != [] :
     #   print (temp1.findall(token), token)
      temp[temp.index(token)] = re.sub(r'([a-zA-Z])\1\1+', r'\1\1', token)
    filtered.append(temp)
    writer.writerow((' '.join(tweet), ';'.join(slang_out)))
text = filtered
print((text))


LMAO
BTW
U
U
U
KISS
U
LOL
U
WTF
WB
LMAO
U
[['Doing', 'a', 'TV', 'note', 'about', 'Haallmarrk', 'movie', ':', ',', '"', '"', 'A', 'Smile', 'as', 'Big', 'as', 'the', 'Moon', ',', '"', '"', 'about', "teacher's", 'fight', 'to', 'take', 'kids', 'to', 'camp', '.', 'Airs', 'ABC', 'Sunday'], ['De', 'La', 'Soul', 'is', 'the', 'shit', 'too', 'no', 'lie'], ['De', 'La', 'Soul', 'is', 'going', 'to', 'be', 'at', 'the', 'Cabooze', 'on', '4/27', '...', 'that', 'is', 'going', 'to', 'be', 'a', 'great', 'show', '...'], ['Wife', '&', 'I', 'made', 'a', 'day', 'trip', 'from', 'Paris', 'just', 'to', 'see', 'The', 'Ladykillers', 'matinee', 'last', 'Wed', '.', 'Absolutely', 'loved', 'the', 'show', '!'], ['Tune', 'into', 'ABC', 'Sunday', ',', 'April', '22nd', '9/8', 'c', 'for', '"', '"', 'Firelight', '"', '"', 'a', 'Hallmark', 'Hall', 'of', 'Fame', 'movie', 'with', 'Cuba', 'Gooding', 'Jr', '.', 'and', 'me', 'as', 'a', 'parole', 'officer', '.'], ['Got', 'fiends', 'throwin', 'up', 'on', 'themselves', 'like', 'Wil

Now for each text, apply data augmentation to generate variations (max 5 variations per text). Use the

following approach to generate variations:

- Replace nouns and verbs with their synonyms. Use wordnet for getting synonyms

(https://www.nltk.org/howto/wordnet.html).

Output the original text and augmented text in a csv file using the following format:

original text,  augmentation1, augmentation2, augmentation3, augmentation4, augmentation5

Fill the empty cell with "None"

In [207]:
#part2
# in this part I replace 1 word at a time with the first synonym if there are no synonyms I go to next applicable word
pos_text = []
for tweet in text:
  pos_text.append(nltk.pos_tag(tweet))



with open('augments.csv', 'w', newline = '') as csvfile:
  writer = csv.writer(csvfile, delimiter = ',', lineterminator='\n')
  writer.writerow(['original text', 'augmentation1', 'augmentation2', 'augmentation3', 'augmentation4', 'augmentation5'])  
  augment_list = []
  i = 0
  for tweet in pos_text:
    augment = []
    augment_list.append((labels[i], text[i]))
    
    augment_count = 1
    for word in tweet:
      temp = copy.deepcopy(text[i])
      if augment_count > 5:
        break
      if 'NN' in word[1] or 'VB' in word[1]:
        syn = wn.synsets(word[0])
        if syn != []:
          augment_count += 1
          syn = syn[0].name().split('.')[0]
          #temp.append(syn)
          temp[temp.index(word[0])] = syn
          augment.append(temp)
          augment_list.append((labels[i], temp))
      # if no more augmentations
      #temp = ['None']
      #print (temp)
      
    #print (augment_count)
    while augment_count < 6:
      #print(text[i])
      augment.append((['None']))
      augment_count += 1
    #print (len(augment))
    writer.writerow((' '.join(text[i]), ' '.join(augment[0]), ' '.join(augment[1]), ' '.join(augment[2]), ' '.join(augment[3]), ' '.join(augment[4])))
    i = i +1



Part 3 - 6
- (3) Now print the size of your augmented dataset, ratio of original/augmented set and label distribution

of the augmented set.

- (3) Generate n-gram (unigram and bigram, trigram) for the augmented dataset. Print them to console

In [208]:
#part3
#print (augment_list)
augmented_with_tags = augment_list
augmented_no_tags = [tweet[1] for tweet in augmented_with_tags]
original_with_tags = []
for i in range (0, len(text)):
  original_with_tags.append((labels[i], text[i]))
#print (augmented_no_tags)

# get distributions
positive = 0
neutral = 0
negative = 0
for tweet in augmented_with_tags:
  if tweet[0] == 1:
    positive += 1
  if tweet[0] == 0:
    neutral += 1
  if tweet[0] == -1:
    negative += 1
print ("length of augmented set:", len(augmented_no_tags))
print ("ratio of original/augmented text:", len(original_with_tags) / len(augmented_with_tags))
print ("Label Distribution :")
print ("positive sentiment:", positive, "neutral sentiment:", neutral, "negative sentiment:", negative)
unigram_sentences = []
bigram_sentences = []
trigram_sentences = []
from nltk.util import ngrams
for tweet in augmented_no_tags:
  unigram_sentences.append(list(ngrams(tweet, 1)))
  bigram_sentences.append(list(ngrams(tweet, 2)))
  trigram_sentences.append(list(ngrams(tweet, 3)))
  #print (list(ngrams(tweet, 2)))
print (unigram_sentences)
print(bigram_sentences)
print(trigram_sentences)

length of augmented set: 10935
ratio of original/augmented text: 0.16945587562871514
Label Distribution :
positive sentiment: 5787 neutral sentiment: 3949 negative sentiment: 1199


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Part 4 - 8
(4) Now compute rank/frequency profile of words of the original corpus and augmented corpus. 

To get the rank/frequency profile, you take the type from the frequency list and replace it with its rank, where the most frequent type is given rank 1, and so forth.

Print the rank/frequency profile of the words in csv files for both corpus as follows:

rank1, freq 1

rank 2, freq 2

......

......

(4) Output the percentage of corpus size made up by the top-10 words for both original and augmented corpus. 

In [209]:
#part4
augmented_token_list = [item for sublist in augmented_no_tags for item in sublist]
original_token_list = [item for sublist in text for item in sublist]
#print (len(augmented_token_list))
from nltk.probability import FreqDist
aug_dist= FreqDist(augmented_token_list)
orig_dist= FreqDist(original_token_list)
#print (aug_dist.freq('Doing'))
with open('Original_Data_Profile.csv', 'w', newline = '') as csvfile:
  writer = csv.writer(csvfile, delimiter = ',', lineterminator='\n')
  writer.writerow(['Rank', 'Frequency', "Word"])
  count = 0
  for word in orig_dist.most_common():
    count += 1
    writer.writerow(("rank "+str(count), orig_dist.freq(word), word))
    #print(("rank"+str(count), orig_dist.freq(word[0]), word[0]))

with open('Augmented_Data_Profile.csv', 'w', newline = '') as csvfile:
  writer = csv.writer(csvfile, delimiter = ',', lineterminator='\n')
  writer.writerow(['Rank', 'Frequency', "Word"])
  count = 0
  for word in aug_dist.most_common():
    count += 1
    writer.writerow(("rank "+str(count), aug_dist.freq(word[0]), word[0]))
    #print(("rank"+str(count), aug_dist.freq(word[0]), word[0]))

aug_top = 0
orig_top = 0

for word in orig_dist.most_common(10):
  orig_top += word[1]
print("original top words", orig_top/len(original_token_list)*100, "%")
for word in aug_dist.most_common(10):
  aug_top += word[1]
print("augmented top words", aug_top/len(augmented_token_list)*100, "%")



2.1062386789671007e-05
original top words 21.223273746933053 %
augmented top words 21.205189772104973 %


Part 5 - 8
Find top-10 bi-grams and tri-grams from positive sentiment samples and top-10 bi-grams and tri-grams from negative sentiment samples using NLTK’s significant collocation approach. Investigate the difference of these n-grams when you try: 

(4) PMI to select significant collocations                                                                  (Review materials from lecture 6)
(4) Maximum likelihood to select significant collocations    (use likelihood ratio. Review materials from lecture 6)       

In [210]:
#part5
from nltk.collocations import *
positive_token_list = data[data['label'] == 1 ]
positive_token_list =[tkn.tokenize(tweet) for tweet in positive_token_list['text']]
positive_token_list = [item for sublist in positive_token_list for item in sublist]

negative_token_list = data[data['label'] == -1 ]
negative_token_list =[tkn.tokenize(tweet) for tweet in negative_token_list['text']]
negative_token_list = [item for sublist in negative_token_list for item in sublist]
#print (original_token_list)
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder1 = BigramCollocationFinder.from_words(positive_token_list)
finder1.apply_freq_filter(4)
print ("positive bigram")
print (finder1.nbest(bigram_measures.pmi, 10))
print (finder1.nbest(bigram_measures.likelihood_ratio, 10) )

trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder2 = TrigramCollocationFinder.from_words(positive_token_list)
finder2.apply_freq_filter(4)
print ("positive trigram")
print (finder2.nbest(trigram_measures.pmi, 10))
print (finder2.nbest(trigram_measures.likelihood_ratio, 10) )

bigram_measures = nltk.collocations.BigramAssocMeasures()
finder1 = BigramCollocationFinder.from_words(negative_token_list)
finder1.apply_freq_filter(3)
print ("negative bigram")
print (finder1.nbest(bigram_measures.pmi, 10))
print (finder1.nbest(bigram_measures.likelihood_ratio, 10) )

trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder2 = TrigramCollocationFinder.from_words(negative_token_list)
finder2.apply_freq_filter(3)
print ("negative trigram")
print (finder2.nbest(trigram_measures.pmi, 10))
print (finder2.nbest(trigram_measures.likelihood_ratio, 10) )

positive bigram
[('vanilla', 'fudge'), ('Kenny', 'Lofton'), ('Mehdi', 'Hasan'), ('Winspear', 'Opera'), ('Australian', 'Tour'), ('Chelsea', 'Flower'), ('Pomona', 'Fairplex'), ('Opera', 'House'), ('Talk', 'Like'), ('Who', 'wants')]
[('"', '"'), ('&', '&'), ('!', '!'), ('going', 'to'), ('will', 'be'), ('to', 'see'), ('Matt', 'Cain'), ('Westfield', 'Parramatta'), ('at', 'the'), ('Ironman', 'Wales')]
positive trigram
[('Winspear', 'Opera', 'House'), ('International', 'Talk', 'Like'), ('Star', 'Wars', 'Weekends'), ('Chelsea', 'Flower', 'Show'), ('De', 'La', 'Soul'), ('Styled', 'To', 'Rock'), ('Diego', 'County', 'Fair'), ('London', 'Grand', 'Prix'), ('San', 'Diego', 'County'), ('Copper', 'River', 'Salmon')]
[('!', '!', '!'), ('going', 'to', 'see'), ('Glory', '"', '"'), ('"', '"', 'Blaze'), ('of', '"', '"'), ('"', '"', 'in'), ('&', '&', 'the'), ('going', 'to', 'be'), ("I'm", 'going', 'to'), ('is', 'going', 'to')]
negative bigram
[('Death', 'Metal'), ('Hat', 'Yai'), ('Kenny', 'Lofton'), ('Resid

Part 6 - 6
  Compute word vectors from this data-set using the following approaches: 

     1) (2) Frequency based    

     2) (2) One-hot

     3) (2) TF-IDF

   Print the word vectors

In [211]:
#part6
# printed word vectors for word in sentence
new_text = list(data['text'])
#print (list(new_text))
count = CountVectorizer()
transformed = count.fit_transform(new_text)
print ("count vectorizer sparse word vectors")
print (transformed)
#print ("worrd", transformed[1])
#print (len(count.vocabulary_))]
#
print ("one hot sparse word vectors")
# adding label encoder so can correctly reshape as per guide
#https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform((original_token_list))
onehot_encoder = OneHotEncoder()
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
#
print ("tfidf sparse word vectors")
tfidf = TfidfVectorizer()
transformed = tfidf.fit_transform(new_text)
print (transformed)

count vectorizer sparse word vectors
  (0, 1870)	1
  (0, 6125)	1
  (0, 4224)	1
  (0, 348)	2
  (0, 2686)	1
  (0, 4029)	1
  (0, 5448)	1
  (0, 625)	2
  (0, 897)	1
  (0, 5881)	1
  (0, 4004)	1
  (0, 5824)	1
  (0, 2284)	1
  (0, 5989)	2
  (0, 5787)	1
  (0, 3345)	1
  (0, 4104)	1
  (0, 5519)	1
  (0, 1156)	1
  (0, 443)	1
  (0, 342)	1
  (0, 5708)	1
  (1, 5881)	1
  (1, 1698)	1
  (1, 3424)	1
  :	:
  (1851, 5282)	1
  (1851, 389)	1
  (1851, 3105)	1
  (1852, 5989)	1
  (1852, 4340)	1
  (1852, 6355)	1
  (1852, 6673)	2
  (1852, 3825)	1
  (1852, 3026)	1
  (1852, 2505)	1
  (1852, 2930)	1
  (1852, 1420)	1
  (1852, 593)	1
  (1852, 2990)	1
  (1852, 6678)	1
  (1852, 3613)	1
  (1852, 5991)	1
  (1852, 4601)	1
  (1852, 4556)	1
  (1852, 1458)	1
  (1852, 5960)	1
  (1852, 660)	1
  (1852, 3465)	1
  (1852, 1830)	1
  (1852, 338)	1
one hot sparse word vectors
  (0, 1097)	1.0
  (1, 3510)	1.0
  (2, 3083)	1.0
  (3, 5377)	1.0
  (4, 3515)	1.0
  (5, 1511)	1.0
  (6, 5311)	1.0
  (7, 327)	1.0
  (8, 13)	1.0
  (9, 1)	1.0
  (10, 1)